In [2]:
import os
import json
from web3 import Account
#from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import requests

In [56]:
# loading environment variables
load_dotenv()


True

In [57]:
# Create a W3 Connection
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_RPOVIDER_URI")))
private_key = os.getenv("PRIVATE_KEY")
contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

In [75]:
# All functions needed to tokenize an NFT
def generate_account(w3,private_key):
    account = Account.privateKeyToAccount(private_key)
    return account

# Set up Pinata Headers
json_headers = {
    "Content-Type":"application/json",
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY")
}

file_headers = {
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY")
}

def convert_data_to_json(content):
    data = {"pinataOptions":{"cidVersion":1}, 
            "pinataContent":content }
    return json.dumps(data)

def pin_file_to_ipfs(data):
    r = requests.post("https://api.pinata.cloud/pinning/pinFileToIPFS",
                      files={'file':data},
                      headers= file_headers)
    print(r.json())
    ipfs_hash = r.json()["IpfsHash"]
    return ipfs_hash

def pin_json_to_ipfs(json):
    r = requests.post("https://api.pinata.cloud/pinning/pinJSONToIPFS",
                      data=json,
                      headers= json_headers)
    print(r.json())
    ipfs_hash = r.json()["IpfsHash"]
    return ipfs_hash

def pin_cert(cert_name, cert_file,**kwargs):
    # Pin certificate picture to IPFS
    ipfs_file_hash = pin_file_to_ipfs(cert_file.getvalue())

    # Build our NFT Token JSON
    token_json = {
      "name": cert_name, 
       "image": f"ipfs.io/ipfs/{ipfs_file_hash}"
    }

    # Add extra attributes if any passed in
    token_json.update(kwargs.items())

    # Add to pinata json to be uploaded to Pinata
    json_data = convert_data_to_json(token_json)

    # Pin the real NFT Token JSON
    json_ipfs_hash = pin_json_to_ipfs(json_data)

    return json_ipfs_hash, token_json

In [76]:
# Pull in Ethereum Account - Used for signing transactions
account = generate_account(w3,private_key)

In [77]:
def load_contract():
    with open(Path("./contracts/compiled/certificate_abi.json")) as file:
        certificate_abi = json.load(file)

    contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

    cert_contract = w3.eth.contract(address=contract_address,
                    abi=certificate_abi)

    return cert_contract      

In [81]:
from array import array

with io.BytesIO() as output:
    image.save(output, format="GIF")
    contents = output.getvalue()

In [79]:
# loading the rinkby contract
contract = load_contract()
customer_account = "0xBb28C656dA091796c67C8787cE9E0801dE20A018"
byteImgIO = io.BytesIO()
byteImg = Image.open("toucan1.png")
byteImg.save(byteImgIO, "PNG")
byteImgIO.seek(0)
byteImg = byteImgIO.read()
# Non test code
dataBytesIO = io.BytesIO(byteImg)

In [88]:
cert_ipfs_hash,token_json = pin_cert(name)

cert_uri = f"ipfs.io/ipfs/{cert_ipfs_hash}"

nonce = w3.eth.getTransactionCount(account.address)
 
tx = contract.functions.mint(customer_account,cert_uri).buildTransaction({
   'chainId':80001,
    'gas':20000000,
    'nonce':nonce
})

AttributeError: 'str' object has no attribute 'getvalue'

In [1]:
from web3 import Web3, HTTPProvider
connection = Web3(HTTPProvider('https://rinkeby.infura.io/v3/dad654cbe1e74e88a12fd7d0c673d000'))
print ("Latest Ethereum block number", connection.eth.blockNumber)

Latest Ethereum block number 11174740


In [4]:
load_dotenv()
link = os.getenv("WEB3_PROVIDER_URI")
link

'https://mainnet.infura.io/v3/575c662560664e68a8fd0cb4ee97e28d'